In [38]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import gcsfs
from sklearn.model_selection import train_test_split
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error

In [3]:
datasets = ["2023-05"]
bucket_name = "prediswiss-parquet-data"

In [4]:
fs_gcs = gcsfs.GCSFileSystem()
dataframes = []
for dataset in datasets:
    path = bucket_name + "/" + dataset + ".parquet"
    table = pq.read_table(path, filesystem=fs_gcs)
    df = table.to_pandas()
    dataframes.append(df)
df = pd.concat(dataframes)

In [85]:
# Specify the parameters
id = 'CH:0542.05'
target = 'flow_11'
date = 'publication_date'

# Filter the DataFrame based on the specified id
filtered_df = df[df['id'] == id].copy()

filtered_df = filtered_df[[date, target]]
filtered_df = filtered_df.sort_values(date)
filtered_df[target] = pd.to_numeric(filtered_df[target], errors='coerce')
filtered_df[date] = pd.to_datetime(filtered_df[date])
filtered_df['flow_11'].dropna(inplace=True)
filtered_df.set_index(date, inplace=True)
filtered_df

,flow_11
publication_date,
2023-05-10 10:57:18.254077+00:00,1380.0
2023-05-10 10:58:18.078285+00:00,1380.0
2023-05-10 10:59:18.203105+00:00,1260.0
2023-05-10 11:00:18.063256+00:00,1560.0
2023-05-10 11:01:18.162343+00:00,1620.0
...,...
2023-05-31 23:52:18.127738+00:00,120.0
2023-05-31 23:53:18.192065+00:00,60.0
2023-05-31 23:54:18.249585+00:00,120.0


In [96]:
print(filtered_df.isna())

AttributeError: 'DataFrame' object has no attribute 'get_group'

In [ ]:
result = adfuller(filtered_df)
print('ADF Statistic: %f' % result[0])
print('p-value: %f' % result[1])

In [82]:
size = 60
train, test = train_test_split(filtered_df, test_size=size/len(filtered_df), shuffle=False)